In [ ]:
from dotenv import load_dotenv
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery,
)

In [ ]:
load_dotenv()

#Define Azure OpenAI credentials
api_key = os.getenv("AZURE_OPENAI_KEY")
api_version = "2023-12-01-preview"
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
engine = os.getenv("EMBEDDING_DEPLOYMENT_NAME")

#Define Azure AI Search credentials
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
credential = AzureKeyCredential(key)

In [ ]:
#create embedding-client using AzureOpenAI credentials
client = AzureOpenAI(api_key=api_key,
    api_version="2023-12-01-preview",
    azure_endpoint =azure_endpoint
    )

### 1) embed a given query

In [ ]:
def query_embedding(client, input, engine):
    client=client
    response= client.embeddings.create(input=input,model=engine)
    query_embedding = response.data[0].embedding 
    return query_embedding

In [ ]:
query="Was versichert die Kfz-Versicherung?"
query_vector=query_embedding(client, query, engine)
print(query_vector)

### 2) perform the search

#### 2.1) Normal Vector search

In [ ]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)

for result in results:
    print(f"Score: {result['@search.score']}")
    print(f"Captions: {result['@search.captions']}")
    print(f"Highlights: {result['@search.highlights']}")
    print(f"Content: {result['line']}\n")
    print(f"Filename: {result['filename']}\n")
    print("######################################")

#### 2.2) Exhaustive KNN search

In [ ]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding", exhaustive=True)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)

for result in results:
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['line']}\n")
    print(f"Filename: {result['filename']}\n")
    print("######################################")


#### 2.3) Hybrid Search - if available in Azure region

In [ ]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding")

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["line", "filename"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)


Semantic ranking of answers

In [ ]:
for result in results:
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['line']}\n") 

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")
